In [2]:
# Import libraries
import xarray as xr
import cmocean as cm
import cartopy.crs as ccrs

import pylab as plt
import numpy as np
# Inline plotting
%matplotlib inline

In [3]:
from dask.distributed import Client

In [4]:
c = Client()
c

/Users/josue/.anaconda3/envs/trackeddy/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63326 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:63327 Dashboard: http://127.0.0.1:63326/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [9]:
dataset_SST_grad = xr.open_dataset('../datasets/SST_grads_timeseries.nc',chunks={'lat':10,'lon':10})

In [11]:
dataset_SST_grad.SST_grad.attrs

{'units': '$^\\circ C$',
 'name': 'SST',
 'long_name': 'Sea Surface Temperature',
 'valid_min': -3.0396215e-12,
 'valid_max': 9.19988e-05,
 'valid_range': array([-3.0396215e-12,  9.1998802e-05], dtype=float32)}